In [1]:
import xarray as xr

In [15]:
data_path = "../data/BCCR/EVAL/raw_data/"
example_file = "ua1000_EUR44_ECMWF-ERAINT_EVAL_r1i1p1_BCCR-WRF381_1hr_1986010100-1986123123.nc"

In [16]:
ds = xr.open_dataset(data_path + example_file)

In [17]:
ds

<xarray.Dataset>
Dimensions:       (rlon: 106, rlat: 103, plev: 1, time: 8616)
Coordinates:
  * rlon          (rlon) float64 -28.21 -27.77 -27.33 ... 17.11 17.55 17.99
  * rlat          (rlat) float64 -23.21 -22.77 -22.33 ... 20.79 21.23 21.67
  * plev          (plev) float64 1e+05
  * time          (time) datetime64[ns] 1986-01-01 ... 1986-12-31T23:00:00
Data variables:
    rotated_pole  |S1 ...
    ua1000        (time, plev, rlat, rlon) float32 ...
    lat           (rlat, rlon) float64 ...
    lon           (rlat, rlon) float64 ...
Attributes: (12/21)
    CDI:                            Climate Data Interface version 1.7.2 (htt...
    institution:                    NORCE Climate, Bjerknes Centre for Climat...
    Conventions:                    CF-1.4
    SWPP_SHIFTTIME:                 shift: 0, left: 0, right: 0
    title:                          CORDEX LUCAS ERA-Interim Run
    experiment:                     evaluation run with reanalysis forcing
    ...                             ...
    project_id:                     CORDEX
    CORDEX_domain:                  EUR44
    product:                        output
    startdate:                      S198601010000
    bccr_run_performance:           Fram Supercomputer at Uninett Sigma
    CDO:                            Climate Data Operators version 1.7.2 (htt...

In [38]:
vars = ["ua"+str(plev) for plev in [1000, 925, 850, 700]]

In [41]:
# open all pressure levels at once and rename variables (e.g., ua925) to ua in all cases (plev is a coordinate, doesn't have to be variable name)
ds_list = [xr.open_mfdataset(data_path + var +"_*2010010100-*.nc").rename({var: "ua"}) for var in vars]

In [42]:
ds_all = xr.merge(ds_list)

In [43]:
ds_all

<xarray.Dataset>
Dimensions:       (rlon: 106, rlat: 103, plev: 4, time: 8760)
Coordinates:
  * rlon          (rlon) float64 -28.21 -27.77 -27.33 ... 17.11 17.55 17.99
  * rlat          (rlat) float64 -23.21 -22.77 -22.33 ... 20.79 21.23 21.67
  * plev          (plev) float64 7e+04 8.5e+04 9.25e+04 1e+05
  * time          (time) datetime64[ns] 2010-01-01 ... 2010-12-31T23:00:00
Data variables:
    rotated_pole  |S1 b''
    ua            (time, plev, rlat, rlon) float32 dask.array<chunksize=(8760, 4, 103, 106), meta=np.ndarray>
    lat           (rlat, rlon) float64 dask.array<chunksize=(103, 106), meta=np.ndarray>
    lon           (rlat, rlon) float64 dask.array<chunksize=(103, 106), meta=np.ndarray>
Attributes: (12/21)
    CDI:                            Climate Data Interface version 1.7.2 (htt...
    institution:                    NORCE Climate, Bjerknes Centre for Climat...
    Conventions:                    CF-1.4
    SWPP_SHIFTTIME:                 shift: 0, left: 0, right: 0
    title:                          CORDEX LUCAS ERA-Interim Run
    experiment:                     evaluation run with reanalysis forcing
    ...                             ...
    project_id:                     CORDEX
    CORDEX_domain:                  EUR44
    product:                        output
    startdate:                      S201001010000
    bccr_run_performance:           Fram Supercomputer at Uninett Sigma
    CDO:                            Climate Data Operators version 1.7.2 (htt...

Testing complete loop

In [76]:
import xarray as xr

# Open wind components at pressure level,
# combine them in one file per year,
# compute wind speeds and
# store everything separately


data_path = "../data/BCCR/"
years = range(1986, 2016)


for experiment in ["EVAL", "FOREST", "GRASS"]:
    data_path_tmp = data_path + experiment + "/"

    for year in years:
        #####################
        # at pressure levels
        #####################
        print(year)
        # ua
        var_names = ["ua" + str(plev) for plev in [1000, 925, 850, 700]]
        # open all pressure levels at once and rename variables (e.g., ua925)
        # to ua in all cases (plev is a coordinate, doesn't have to be variable name)
        ds_list = [
            xr.open_mfdataset(
                data_path_tmp + "raw_data/" + var + "_*" + str(year) + "010100-*.nc"
            ).rename({var: "U"})
            for var in vars
        ]
        ds_ua = xr.merge(ds_list)
        ds_ua.to_netcdf(data_path_tmp + "U/U_" + str(year) + ".nc")

        # va
        var_names = ["va" + str(plev) for plev in [1000, 925, 850, 700]]
        # open all pressure levels at once and rename variables (e.g., ua925)
        # to ua in all cases (plev is a coordinate, doesn't have to be variable name)
        ds_list = [
            xr.open_mfdataset(
                data_path_tmp + "raw_data/" + var + "_*" + str(year) + "010100-*.nc"
            ).rename({var: "V"})
            for var in vars
        ]
        ds_va = xr.merge(ds_list)
        ds_va.to_netcdf(data_path_tmp + "V/V_" + str(year) + ".nc")

        # s
        ds_s = (ds_ua["U"] ** 2 + ds_va["V"] ** 2) ** (1.0 / 2)
        ds_s = ds_s.to_dataset(name="S")
        ds_s["S"].attrs = {
            "standard_name": "wind_speed",
            "long_name": "Wind speed",
            "units": "m s-1",
            "grid_mapping": "rotated_pole",
            "cell_methods": "time: point",
        }
        ds_s.to_netcdf(data_path_tmp + "S/S_" + str(year) + ".nc")

        # zg
        var_names = ["zg" + str(plev) for plev in [1000, 925, 850, 700]]
        # open all pressure levels at once and rename variables (e.g., ua925)
        # to ua in all cases (plev is a coordinate, doesn't have to be variable name)
        ds_list = [
            xr.open_mfdataset(
                data_path_tmp + "raw_data/" + var + "_*" + str(year) + "010100-*.nc"
            ).rename({var: "zg"})
            for var in vars
        ]
        ds_zg = xr.merge(ds_list)
        ds_zg.to_netcdf(data_path_tmp + "ZG/ZG_" + str(year) + ".nc")

        #####################
        # at 10M and 100M
        #####################

        folder_dictionary = {"100m": "100", "s": "10"}

        for var_name_ending in ["100m", "s"]:
            ds_ua = xr.open_mfdataset(
                data_path_tmp
                + "raw_data/ua"
                + var_name_ending
                + "_*"
                + str(year)
                + "010100-*.nc"
            )
            ds_ua.to_netcdf(
                data_path_tmp
                + "U_"
                + folder_dictionary[var_name_ending]
                + "/U_"
                + folder_dictionary[var_name_ending]
                + "_"
                + str(year)
                + ".nc"
            )
            ds_va = xr.open_mfdataset(
                data_path_tmp
                + "raw_data/va"
                + var_name_ending
                + "_*"
                + str(year)
                + "010100-*.nc"
            )
            ds_va.to_netcdf(
                data_path_tmp
                + "V_"
                + folder_dictionary[var_name_ending]
                + "/V_"
                + folder_dictionary[var_name_ending]
                + "_"
                + str(year)
                + ".nc"
            )

            # s
            ds_s = (ds_ua["ua"+var_name_ending] ** 2 + ds_va["va"+var_name_ending] ** 2) ** (1.0 / 2)

            ds_s = ds_s.to_dataset(name="S_" + folder_dictionary[var_name_ending])
            ds_s["S_" + folder_dictionary[var_name_ending]].attrs = {
                "standard_name": folder_dictionary[var_name_ending] + "m wind_speed",
                "long_name": "Wind speed at "
                + folder_dictionary[var_name_ending]
                + "m",
                "units": "m s-1",
                "grid_mapping": "rotated_pole",
                "cell_methods": "time: point",
            }
            ds_s.to_netcdf(
                data_path_tmp
                + "S_"
                + folder_dictionary[var_name_ending]
                + "/S_"
                + folder_dictionary[var_name_ending]
                + "_"
                + str(year)
                + ".nc"
            )


1986
1987
1988
1989


KeyboardInterrupt: 